In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.io
from operator import itemgetter
import pandas as pd
from sklearn.cross_validation import train_test_split

import csv
import nupic

In [2]:
traindf = pd.read_csv('./lab_data/24chan/24chan_train1.csv', header = 0, skiprows = [1,2])
testdf = pd.read_csv('./lab_data/24chan/24chan_test1.csv', header = 0, skiprows = [1,2])

print traindf.shape
print traindf.head()

(67480, 25)
        field1       field2       field3      field4      field5      field6  \
0 -3025.031006 -3898.197510 -5941.647461 -143.265717  223.767761 -896.394226   
1 -3025.013184 -3900.128662 -5938.768555 -139.367584  225.287674 -896.894958   
2 -3025.192139 -3902.274414 -5937.176758 -138.205292  225.430725 -896.054504   
3 -3022.474121 -3898.537109 -5932.116699 -136.184692  228.291748 -893.175598   
4 -3019.648682 -3890.043457 -5928.969238 -131.589172  232.779984 -891.941772   

         field7       field8        field9      field10       ...        \
0 -11081.028320 -9336.876953 -50944.109375 -8007.466797       ...         
1 -11080.598633 -9336.948242 -50917.480469 -8008.665039       ...         
2 -11080.205078 -9336.787109 -50951.457031 -8010.721191       ...         
3 -11071.533203 -9329.348633 -50996.410156 -8014.029297       ...         
4 -11068.529297 -9326.237305 -50985.449219 -8013.957520       ...         

        field16      field17      field18     field19   

In [3]:
SWARM_CONFIG = {
    "includedFields": [
        {
            'fieldName': u'field1',
            #'name': u'field1',
            #'n': 521,
            'fieldType': 'float',
            'minValue': 100000.0,
            'minValue': -100000.0
            #'w': 21
        },
        {
            'fieldName': u'field2',
            'fieldType': 'float',
            'minValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field3',
            'fieldType': 'float',
            'minValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field4',
            'fieldType': 'float',
            'minValue': 100000.0,
            'minValue': -100000.0
            },
        {
            'fieldName': u'field5',
            'fieldType': 'float',
            'minValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field6',
            'fieldType': 'float',
            'minValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field7',
            'fieldType': 'float',
            'minValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field8',
            'fieldType': 'float',
            'minValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field9',
            'fieldType': 'float',
            'minValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field10',
            'fieldType': 'float',
            'minValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field11',
            'fieldType': 'float',
            'minValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field12',
            'fieldType': 'float',
            'minValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field13',
            'fieldType': 'float',
            'minValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field14',
            'fieldType': 'float',
            'minValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field15',
            'fieldType': 'float',
            'minValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field16',
            'fieldType': 'float',
            'minValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field17',
            'fieldType': 'float',
            'minValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field18',
            'fieldType': 'float',
            'minValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field19',
            'fieldType': 'float',
            'maxValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field20',
            'fieldType': 'float',
            'maxValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field11',
            'fieldType': 'float',
            'maxValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field12',
            'fieldType': 'float',
            'maxValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field13',
            'fieldType': 'float',
            'maxValue': 100000.0,
            'minValue': -100000.0},
        {
            'fieldName': u'field14',
            'fieldType': 'float',
            'maxValue': 100000.0,
            'minValue': -100000.0},
        {
            #'classifierOnly': True,
            'fieldName': u'classification',
            'fieldType': 'int'}
    ],
    "streamDef": {
        "info": "classification_24channels",
        "version": 1,
        "streams": [
            {
                "info": "sine.csv",
                "source": "file:///home/polina/Documents/hse/course_work/lab_data/24chan/24chan_train1.csv",
                "columns": [
                  "*"
                ]
            }
        ]
    },
    "inferenceType": "NontemporalClassification",
    "inferenceArgs": {
        "predictionSteps": [
          0
        ],
        "predictedField": u'classification'
    },
    "swarmSize": "medium"
}

In [4]:
from nupic.swarming import permutations_runner
from nupic.swarming.permutations_runner import DEFAULT_OPTIONS
import nupic.swarming

In [5]:
def swarm_over_data():
    return nupic.swarming.permutations_runner.runWithConfig(SWARM_CONFIG, {'maxWorkers': 8, 'overwrite': True})


input_file = "./lab_data/24chan/24chan_train1.csv"
model_params = swarm_over_data()

Generating experiment files in directory: /home/polina/Documents/hse/course_work...
Writing  313 lines...
Writing  114 lines...
done.
None
Successfully submitted new HyperSearch job, jobID=1008
##>> UPDATED WORKER STATE: 
{   u'activeSwarms': [   u'modelParams|sensorParams|encoders|field17',
                         u'modelParams|sensorParams|encoders|field18',
                         u'modelParams|sensorParams|encoders|field19',
                         u'modelParams|sensorParams|encoders|field8',
                         u'modelParams|sensorParams|encoders|field9',
                         u'modelParams|sensorParams|encoders|field6',
                         u'modelParams|sensorParams|encoders|field7',
                         u'modelParams|sensorParams|encoders|field4',
                         u'modelParams|sensorParams|encoders|field5',
                         u'modelParams|sensorParams|encoders|field2',
                         u'modelParams|sensorParams|encoders|field3',
     

In [6]:
from nupic.frameworks.opf.modelfactory import ModelFactory
model = ModelFactory.create(model_params)
model.enableInference({"predictedField": 'classification'})

In [10]:
model.save("/home/polina/Documents/hse/course_work/lab_data/24chan/model24_1")
#model.load("/home/polina/Documents/hse/course_work/lab_data/24chan/model24_1")

In [7]:
output = zip(np.asarray(testdf['classification']), np.zeros(testdf.shape[0], dtype = int))
output = pd.DataFrame(output)

In [8]:
input_dict = testdf.to_dict(orient = "index")
for k in input_dict.keys():
    row = input_dict[k]
    #result = model.run({'classification': int(row['classification'])})
    result = model.run(row)
    output[1][k] = result.inferences['multiStepBestPredictions'][0]
    #print row['classification'], result.inferences['multiStepBestPredictions'][0]
    
print output.head()

   0    1
0  1    0
1  1    0
2  1  127
3  1  127
4  1  127


In [9]:
#print output.head()
print np.sum(np.array(output[1]) != np.array(testdf['classification']))
print output.shape
print np.array(output[1]) != np.array(testdf['classification'])

22175
(33237, 2)
[ True  True  True ..., False False False]


In [ ]:
from nupic.swarming import permutations_runner

def f(a):
    return a